In [ ]:
!pip install transformers datasets evaluate sacrebleu
!pip install 'huggingface_hub[tensorflow]'
!pip install huggingface_hub
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer , DataCollatorForSeq2Seq , AdamWeightDecay , TFAutoModelForSeq2SeqLM
import evaluate
import numpy as np
from transformers import DataCollatorForSeq2Seq
import numpy as np
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback


In [ ]:
notebook_login()

In [ ]:
def load_dataset_huggin(argument):
  "Load the subset from datasets library"
  books = load_dataset("opus_books", argument)
  return books

In [ ]:
english_french = load_dataset_huggin("en-fr")
english_hungarian = load_dataset_huggin("en-hu")
english_netherlands = load_dataset_huggin("en-nl")
english_russian = load_dataset_huggin("en-ru")
english_polish = load_dataset_huggin("en-pl")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
### We can select another languages to translate them, we based this on another languages that 
### have more rows so the finetunning makes more sense

In [ ]:
def split_train_test(argument):
  "Load the subset from datasets library"
  books = argument["train"].train_test_split(test_size=0.2)
  return books

In [ ]:
books = split_train_test(english_french)

In [ ]:
books["train"][0]

{'id': '104793',
 'translation': {'en': 'They were then careful to cover their eyes and mouths with handkerchiefs, for they ran the risk of being blinded and suffocated.',
  'fr': "Ils avaient soin alors d'appliquer un mouchoir sur leurs yeux et leur bouche, car ils couraient le risque d'être aveuglés et étouffés."}}

In [ ]:
checkpoint = "t5-small"

In [ ]:

def load_tokenizer():
  "Load the tokenizer to process the language pairs"
  checkpoint = "t5-small"
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  return tokenizer

tokenizer = load_tokenizer()

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French:"

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
## Future, create validation dataset

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [ ]:
push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_opus_books_model",
    tokenizer=tokenizer,
)

/content/my_awesome_opus_books_model is already a clone of https://huggingface.co/SoyGema/my_awesome_opus_books_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

Epoch 1/3
 917/6354 [===>..........................] - ETA: 29:34:08 - loss: 2.0740

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [ ]:
from transformers import pipeline

In [ ]:
translator = pipeline("translation", model="my_awesome_opus_books_model")

In [ ]:
translator(text)